In [ ]:
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
import svg_render_helpers as rrh

#### directory and file hierarchy

In [11]:
# directory & file hierarchy
analysis_dir = os.getcwd()
tracing_ref_pixel = os.path.join(analysis_dir, 'tracing_ref_pixel')
tracing_ref_svg = os.path.join(analysis_dir, 'tracing_ref_svg')
data_dir = os.path.join(analysis_dir,"trace_test")

canvas_side = 432 # image size
rows, cols = canvas_side, canvas_side # the size of a given drawing image

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['cdm_run_v3']

#### Get SVG data

In [ ]:
# test on a specific square with one stroke
imrecs = coll.find({'$and':[{'category':'square'},{'dataType':'finalImage'}, {'sessionId':'cdm_run_v31525889224904'}]}).sort('time')
print imrecs.count()

## retrieve example square tracing from db
imrec = imrecs[0] 

In [ ]:
reload(rsh)
## get list of vertices and codes for tracing 
try:
    ## get the corresponding stroke records for this tracing
    stroke_recs = coll.find({'$and': [{'sessionId':imrec['sessionId']}, 
                                      {'dataType':'stroke'},
                                      {'trialNum': imrec['trialNum']}]}).sort('time')

    print 'Number of strokes in this tracing: {}'.format(stroke_recs.count())    
    
    ## check to make sure that there is at least one stroke! 
    if stroke_recs.count()>0:

        ## now make an svg list!
        svg_list = rsh.make_svg_list(stroke_recs)

        ## get verts and codes in all strokes
        ## both verts_list and codes_list are 2D list. Each sublist represent a stroke
        verts_list, codes_list = rsh.get_verts_and_codes(svg_list)
        
        ## Convert multiple strokes into one stroke
        
        
except Exception as e:
    print 'Oops, something went wrong! Here is the error:'  
    print e
    pass        

#### Tracing/Copying Error Calculation 

In [62]:
# change names. only for testing purpose
for index, imgfile in enumerate(os.listdir(data_dir)):
    new_filename = 'test{:02}.png'.format(index)
    os.rename(os.path.join(data_dir,imgfile), os.path.join(data_dir,new_filename))

In [29]:
reload(rsh)
# get tracings from the data folder
#for file in os.listdir(data_dir):

# read in tracing and reference files
fpath = os.path.join(data_dir, 'test_50.png')
img_draw = cv2.imread(fpath)
img_draw = rsh.white_color_to_num(img_draw)

square_ref_file = os.path.join(tracing_ref_pixel, 'resize_square.png')
img_ref = cv2.imread(square_ref_file)
img_ref = rsh.black_color_to_num(img_ref)

In [57]:
reload(rsh)
loss, final_draw, weights, bias = rsh.minimize_shape_error(img_ref, img_draw)
print loss
print weights
print bias

tensor(0.0395, requires_grad=True)
Parameter containing:
tensor([[1., 0.],
        [0., 1.]], requires_grad=True)
Parameter containing:
tensor([0., 0.], requires_grad=True)


#### preprocessing of tracing and visualization (if desired)


In [ ]:
reallyRun = 1
if reallyRun:
    rsh.plot_shape(verts_list, codes_list, canvas_side)
    #rsh.plot_stroke(verts_list[1], codes_list[1], canvas_side)

#### Prepare human rating

In [59]:
data = pd.DataFrame(os.listdir(data_dir))
data.columns = ['image_id']
data.to_csv(os.path.join(analysis_dir,'performance.csv'))